## Creating the GeoTIFF overlayed with the Building Footprint file
### demonstrated for a single tile

In [2]:
#Suppress warnings
import warnings
warnings.filterwarnings('ignore')

#For GeoTIFF images
import rasterio
from rasterio.plot import show
from osgeo import gdal, ogr, osr
import geopandas as gpd

#For Visualisation
from matplotlib import pyplot as plt
import matplotlib.image as img
from matplotlib.pyplot import figure
from PIL import Image

# Others
import os
import shutil
import zipfile
from tqdm import tqdm

In [3]:
path_to_shapefile = "./provided_footprints/building_footprint_roi_challenge.shp"
path_to_pre_event_image = './Pre_Event_San_Juan.tif'
path_to_post_event_image = './Post_Event_San_Juan.tif'

In [4]:
output_pre_event_raster = './pre_event_footprint.tif'
output_post_event_raster = './post_event_footprint.tif'

In [12]:
input_shp = ogr.Open('./provided_footprints/building_footprint_roi_challenge.shp')
source_layer = input_shp.GetLayer()
defn = source_layer.GetLayerDefn()
column_names = []
for n in range(defn.GetFieldCount()):
    fdefn = defn.GetFieldDefn(n)
    column_names.append(fdefn.name)
column_names

['latitude', 'longitude', 'area_in_me', 'confidence', 'full_plus_']

In [13]:
# gdf = gpd.read_file(path_to_shapefile)
# gdf.head()

In [26]:
def generate_tiles(input_file, shapefile, output_dir,grid_x,grid_y):
    ds = gdal.Open(input_file)

    # Get image size and number of bands
    width = ds.RasterXSize
    height = ds.RasterYSize
    num_bands = ds.RasterCount

    # Calculate number of tiles in each dimension
    num_tiles_x = (width // grid_x)
    num_tiles_y = (height // grid_y)

    print(f"Total number of tiles: {num_tiles_x * num_tiles_y}")

    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Open the shapefile and get the layer
    shapefile_ds = ogr.Open(shapefile)
    layer = shapefile_ds.GetLayer()

    # Iterate over each tile and save as a separate TIFF image
    for i in tqdm([8]):
        for j in [21]:
            x_offset = i *  grid_x
            y_offset = j *  grid_y

            tile_width = min(grid_x, width - x_offset)
            tile_height = min(grid_y, height - y_offset)

            tile = []
            for band in range(1, num_bands + 1):
                tile_data = ds.GetRasterBand(band).ReadAsArray(x_offset, y_offset, tile_width, tile_height)
                tile.append(tile_data)

            # Create output filename
            output_file = os.path.join(output_dir, f"tile_{i}_{j}.tif")
                        
            # Create an output TIFF file with same CRS and band values range
            driver = gdal.GetDriverByName("GTiff")
            options = ['COMPRESS=DEFLATE', 'PREDICTOR=2', 'TILED=YES']
            out_ds = driver.Create(output_file, tile_width, tile_height, num_bands, 
                       ds.GetRasterBand(1).DataType, options=options)
            # out_ds = driver.Create(output_file, tile_width, tile_height, num_bands, ds.GetRasterBand(1).DataType)

            # Set the geotransform
            geotransform = list(ds.GetGeoTransform())
            geotransform[0] = geotransform[0] + x_offset * geotransform[1]
            geotransform[3] = geotransform[3] + y_offset * geotransform[5]
            out_ds.SetGeoTransform(tuple(geotransform))

            # Set the projection
            out_ds.SetProjection(ds.GetProjection())

            # Write each band to the output file
            for band in range(1, num_bands + 1):
                out_band = out_ds.GetRasterBand(band)
                out_band.WriteArray(tile[band - 1])
                
            gdal.RasterizeLayer(out_ds, [1], layer, burn_values=[255], options=["ALL_TOUCHED=TRUE"])

            # Close the output file
            out_ds = None

    print("Tiles generation completed.")

In [27]:
!rm -rf "./Pre_Event_Grids_In_TIFF"
input_file = path_to_pre_event_image
shape_file = path_to_shapefile
output_dir = "./Pre_Event_Grids_In_TIFF"
grid_x = 512
grid_y = 512
generate_tiles(input_file, shape_file, output_dir,grid_x,grid_y)

Total number of tiles: 10730


100%|██████████| 1/1 [00:00<00:00, 22.56it/s]

Tiles generation completed.
